In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
import urllib.request as req

base_link = 'https://www.easylaw.go.kr/'

# 가정법률, 아동*청소년/교육, 부동산/임대차, ... 카테고리 범위 : 1 (가정법률) ~ 18 (국가 및 지자체)
total_categories = 18
categories = [
    '가정법률', '아동·청소년/교육', '부동산/임대차', '금융/금전', '사업', '창업', 
    '무역/출입국', '소비자', '문화/여가생활', '민형사/소송', '교통/운전', '근로/노동',
    '복지', '국방/보훈', '정보통신/기술', '환경/에너지', '사회안전/범죄', '국가 및 지자체'
]

In [5]:
# 각 페이지마다 텍스트 모으기
def select_data(href): 
    res = req.urlopen(href)
    soup = BeautifulSoup(res, 'html.parser')
    paragraph = []
    
    div_list = soup.select('div#ovDiv > div > div')
    for idx, div in enumerate(div_list):
        if len(div.text) <= 0 or div.text.startswith('인쇄체크'):
            continue
        
        # 텍스트 처리
        if div.table:  # table tag일 경우
            tr_text = []
        
            tr = div.table.tbody.tr   # 행마다 조회
            while tr:
                tr_text.append(tr.text.strip().replace("\n\n\n", " "))
            
                tr = tr.next_sibling.next_sibling
                if tr is None:
                    break
                
            paragraph.append('\n'.join(tr_text))
        else:  
            tmp_text = div.text.replace("\r", "").replace("\n", "").strip()
            tmp_text = tmp_text.replace("      ", " ")
            paragraph.append(tmp_text)
    
    return '\n'.join(paragraph)

In [6]:
# 각 카테고리의 리스트마다 텍스트파일 생성
def gather_data_from_page(main_html, page_name):  
    soup = BeautifulSoup(main_html, 'html.parser')
    
    # 목차 - 링크 수집
    a_list = soup.select('#lnb > div.list > div > ul > li > ul > li > a')
    paragraph = []
    print("size : ", len(a_list))

    # 문단 수집
    for a in a_list:
        href = base_link + a.attrs['href']
        paragraph.append(select_data(href))
    paragraph = '\n'.join(paragraph)
    # print('paragraph : ', paragraph)
    
    # [page_name].txt 파일 생성
    txt_name = './file/' + page_name
    with open(txt_name + '.txt', 'w', encoding='utf-8') as f:
        f.write(paragraph)

In [7]:
def search_pages(category_idx):
    driver = webdriver.Chrome("C:/Users/jeewo/Downloads/chromedriver_win32/chromedriver.exe")
    driver.get('https://www.easylaw.go.kr/CSP/CsmSortRetrieveLst.laf?sortType=cate&csmAstSeq=' + str(category_idx))
    
    # title_page : 19
    title_page = len(driver.find_elements_by_css_selector('#contents > div.img_list > ul > li'))
    for page in range(1, title_page + 1):
        try:
            # 리스트에서 제목 click
            driver.find_element_by_xpath('//*[@id="contents"]/div[5]/ul/li[' + str(page) + ']/a').click()
    
            # 제목 
            page_name = driver.find_element_by_class_name('page_title.another_title2').text
            page_name = page_name.replace('Ⅰ', '1').replace('Ⅱ', '2')
            hangul = re.compile('[^ ()ㆍ0-9ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든글자
            page_name = hangul.sub('', page_name)
            print("page : ", page, " - ", page_name, end='   ')
            
            # 상세히 보기 click
            driver.find_element_by_class_name('btn_modal').click()
            main_html = driver.page_source
            
            # 데이터 수집 --> [page_name].txt 파일로 만들기
            gather_data_from_page(main_html, page_name)
        
            driver.back()
            driver.back()
        except:
            print("에러")
            break

    driver.close()

In [8]:
for category_idx in range(1, total_categories + 1): # 1~18
    print("{}번째 {}".format(categories[category_idx-1], str(category_idx)))
    search_pages(category_idx)

가정법률번째 1
page :  1  -  가족관계 등록   size :  24
page :  2  -  결혼준비자   size :  33
page :  3  -  상속   size :  23
page :  4  -  유언   size :  20
page :  5  -  이혼   size :  51
page :  6  -  입양   size :  29
page :  7  -  장사(장례ㆍ매장ㆍ화장ㆍ자연장)   size :  32
page :  8  -  재혼   size :  19
page :  9  -  태아 및 신생아   size :  24
page :  10  -  후견제도   size :  21
아동·청소년/교육번째 2
page :  1  -  가수(아이돌)   size :  37
page :  2  -  근로청소년   size :  11
page :  3  -  불량식품   size :  16
page :  4  -  실종아동   size :  10
page :  5  -  아동ㆍ청소년 대상 성범죄   size :  17
page :  6  -  아동학대   size :  9
page :  7  -  어린이 생활건강   size :  15
page :  8  -  어린이 생활안전   size :  21
page :  9  -  어린이 식품안전   size :  13
page :  10  -  어린이집 설치   size :  10
page :  11  -  어린이집 운영   size :  15
page :  12  -  영유아 보육   size :  18
page :  13  -  영유아교육   size :  26
page :  14  -  외국인유학생   size :  25
page :  15  -  청소년유해환경   size :  21
page :  16  -  청소년의 인터넷 이용하기   size :  17
page :  17  -  학교 밖 청소년   size :  15
page :  18  -  학교폭력   size :  18
page :  19